Empezamos con los imputs y leyendo el csv

In [72]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


df = pd.read_csv('diabetes_prediccion_normalizado.csv')

Dividimos los valores de entrada y salida(x, y), dividimos para entrenamiento y prueba y los transponemos para no tener problemas luego con los calculos, luego inicializamos de forma random (controlada) los pesos y sesgos de las capas ocultas y de salida

In [73]:
X = df.drop(columns=['diabetes']).values
y = df['diabetes'].values.reshape(-1, 1)

XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=1/3, random_state=42)
XTrain = XTrain.T
XTest = XTest.T
yTrain = yTrain.T
yTest = yTest.T

n = XTrain.shape[0]
nHidden = 6
nOutputs = 1

np.random.seed(50)

WHidden = np.random.randn(nHidden, n)
bHidden = np.random.randn(nHidden, 1)

WOutput = np.random.randn(nOutputs, nHidden)
bOutput = np.random.randn(nOutputs, 1)

print("W1 shape:", WHidden.shape)
print("W2 shape:", WOutput.shape)
print("b1 shape:", bHidden.shape)
print("b2 shape:", bOutput)

W1 shape: (6, 12)
W2 shape: (1, 6)
b1 shape: (6, 1)
b2 shape: [[-0.19904969]]


Creamos las funciones de activacion relu y logistic, ademas de la funcion forwardProp

In [74]:
relu = lambda x: np.maximum(x, 0) #Para la capa oculta
logistic = lambda x: 1 / (1 + np.exp(-x)) #Para la capa de salida

reluDerivada = lambda x: x > 0
logisticDerivada = lambda x: np.exp(-x) / (1 + np.exp(-x)) ** 2

def forwardProp(X): # Forward propagation con una capa oculta y una capa de salida
    Z1 = WHidden @ X + bHidden
    A1 = relu(Z1)
    Z2 = WOutput @ A1 + bOutput
    A2 = logistic(Z2)
    return Z1, A1, Z2, A2

Llamamos la forwardProp con los datos de testeo, convertimos las predicciones en binarios y los aplanamos a un array, tambien aplanamos los outputs reales para que tengan la misma dimension. Luego comparamos las predicciones con los outputs reales y calculamos la presicion de nuestro modelo

In [75]:
print("XTest:", XTest.shape)
print("WHidden:", WHidden.shape)

testeoPredicciones = forwardProp(XTest)[3]

prediccionesProcesadas = (testeoPredicciones >= 0.5).flatten().astype(int)
datosRealesProcesados = yTest.flatten().astype(int)


comparacion = np.equal(prediccionesProcesadas, datosRealesProcesados)
precision = np.sum(comparacion.astype(int)) / yTest.size

print(f"Número de muestras de prueba: {yTest.size}")
print(f"Número de predicciones correctas: {np.sum(comparacion.astype(int))}")
print("Precisión:", precision)

XTest: (12, 33328)
WHidden: (6, 12)
Número de muestras de prueba: 33328
Número de predicciones correctas: 21294
Precisión: 0.638922227556409


Utilizamos el backwardPropagation de forma totalmente standar y fiel a la teoria de nuestros apuntes, haciendo las derivadas parciales por proceso

In [76]:
def backwardProp(Z1, A1, Z2, A2, X, Y):
    dC_dA2 = 2 * A2 - 2 * Y
    dA2_dZ2 = logisticDerivada(Z2)
    dZ2_dA1 = WOutput
    dZ2_dW2 = A1
    dZ2_dB2 = 1
    dA1_dZ1 = reluDerivada(Z1)
    dZ1_dW1 = X
    dZ1_dB1 = 1

    dC_dW2 = dC_dA2 @ dA2_dZ2 @ dZ2_dW2.T

    dC_dB2 = dC_dA2 @ dA2_dZ2 * dZ2_dB2

    dC_dA1 = dC_dA2 @ dA2_dZ2 @ dZ2_dA1

    dC_dW1 = dC_dA1 @ dA1_dZ1 @ dZ1_dW1.T

    dC_dB1 = dC_dA1 @ dA1_dZ1 * dZ1_dB1

    return dC_dW1, dC_dB1, dC_dW2, dC_dB2

Damos un ritmo de apredizaje y empezamos a entrenar nuestro modelo con descenso de gradiente estocastico, que es tambien la forma que vimos durante la cursada, ajustando los pesos y sesgos con cada iteracion

In [77]:
L = 0.01

for i in range(10000):

    idx = np.random.choice(n, 1, replace=False)
    X_sample = XTrain[:, idx]
    Y_sample = yTrain[:, idx]

    Z1, A1, Z2, A2 = forwardProp(X_sample)

    dW1, dB1, dW2, dB2 = backwardProp(Z1, A1, Z2, A2, X_sample, Y_sample)

    WHidden -= L * dW1
    bHidden -= L * dB1
    WOutput -= L * dW2
    bOutput -= L * dB2

Volvemos a utilizar nuestro codigo de presicion para ver como le va a nuestra red una vez ya entrenada, mirando los resultados con los datos de entrenamiento y de testeo para poder compararlos

In [78]:
entrenamientoPredicciones = forwardProp(XTrain)[3]

prediccionesEntrenamientoProcesadas = (entrenamientoPredicciones >= 0.5).flatten().astype(int)
datosRealesEntrenamientoProcesados = yTrain.flatten().astype(int)


comparacion = np.equal(prediccionesEntrenamientoProcesadas, datosRealesEntrenamientoProcesados)
precision = np.sum(comparacion.astype(int)) / yTrain.size

print(f"Número de muestras de entrenamiento: {yTrain.size}")
print(f"Número de predicciones correctas: {np.sum(comparacion.astype(int))}")
print("Precisión:", precision)

print("-------------------------------------------------------------------------")

testeoPredicciones = forwardProp(XTest)[3]

prediccionesProcesadas = (testeoPredicciones >= 0.5).flatten().astype(int)
datosRealesProcesados = yTest.flatten().astype(int)


comparacion = np.equal(prediccionesProcesadas, datosRealesProcesados)
precision = np.sum(comparacion.astype(int)) / yTest.size

print(f"Número de muestras de prueba: {yTest.size}")
print(f"Número de predicciones correctas: {np.sum(comparacion.astype(int))}")
print("Precisión:", precision)

Número de muestras de entrenamiento: 66654
Número de predicciones correctas: 60914
Precisión: 0.9138836378911993
-------------------------------------------------------------------------
Número de muestras de prueba: 33328
Número de predicciones correctas: 30366
Precisión: 0.9111257801248199
